In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from nltk.stem.porter import PorterStemmer

/home/marktanios/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
file_1 = pd.read_csv("twitter-2013train.txt", sep='\t', header=None)
file_2 = pd.read_csv("twitter-2015train.txt", sep='\t', header=None)
file_3 = pd.read_csv("twitter-2016train.txt", sep='\t', header=None)
Stop_Words = pd.read_csv("stopwords.txt",header=None)
all_files = pd.concat([file_1, file_2, file_3], ignore_index=True)
all_files.head()
All_Data = all_files.drop(all_files.columns[[0]], axis=1)
All_Data.head()
Stop_Words = Stop_Words.iloc[3:]
Stop_Words.columns = ['Words']
StopWords = Stop_Words['Words'].tolist()
All_Data.columns = ['Polarity','Tweet']
for a in StopWords:
    a = unicode(a, "utf-8")

In [3]:
def PreProcess(line):
    line = line.lower()
    curr = ""
    for word in line.split():
        word = re.sub(r'\\[u][0-9]{1,}', '', word)
        word = re.sub(r'@[a-zA-Z0-9_.-]{1,}','',word)
        word = re.sub(r"[^a-zA-Z0-9]+", ' ', word)
        if word not in StopWords:
            curr = curr + word +" "
    curr = curr.strip()
    curr = PorterStemmer().stem(curr)
    curr = curr.strip()
    return curr

def CountEmoticons (line):
    postive = line.count(':)')
    postive = postive + line.count(':D')
    postive = postive + line.count('=)')
    postive = postive + line.count(':*')
    postive = postive + line.count(':-)')
    postive = postive + line.count('<3')
    negative = line.count(':(')
    negative = negative + line.count(':_(')
    negative = negative + line.count(':''(')
    negative = negative + line.count('=(')
    negative = negative + line.count(':-(')
    negative = negative + line.count(':S')
    return postive - negative

def CountHashtags (line):
    count = line.count('#')
    return count

def CountURLS (line):
    count = line.count('http')
    return count

def CountMentions (line):
    count = line.count('#')
    return count
                                         
def CountExclamationMark(line):
    count = line.count('!')
    return count

In [4]:
All_Data['Emoticons'] = All_Data['Tweet'].apply(CountEmoticons)
All_Data['HashTags'] = All_Data['Tweet'].apply(CountHashtags)
All_Data['Mentions'] = All_Data['Tweet'].apply(CountMentions)
All_Data['URLS'] = All_Data['Tweet'].apply(CountURLS)
All_Data['EM'] = All_Data['Tweet'].apply(CountExclamationMark)
All_Data.head()

,Polarity,Tweet,Emoticons,HashTags,Mentions,URLS,EM
0,positive,Gas by my house hit $3.39!!!! I\u2019m going t...,1,0,0,0,4
1,negative,Theo Walcott is still shit\u002c watch Rafa an...,0,0,0,0,0
2,negative,its not that I\u2019m a GSP fan\u002c i just h...,0,0,0,0,0
3,negative,Iranian general says Israel\u2019s Iron Dome c...,0,0,0,0,0
4,neutral,Tehran\u002c Mon Amour: Obama Tried to Establi...,0,0,0,1,0


In [5]:
All_Data['Tweet'] = All_Data['Tweet'].apply(PreProcess)

In [6]:
All_Data.head()

,Polarity,Tweet,Emoticons,HashTags,Mentions,URLS,EM
0,positive,gas house hit 3 39 im chapel hill sat,1,0,0,0,4
1,negative,theo walcott shitc watch rafa johnny deal satu...,0,0,0,0,0
2,negative,im gsp fanc hate nick diaz wait februari,0,0,0,0,0
3,negative,iranian general israels iron dome deal missile...,0,0,0,0,0
4,neutral,tehranc mon amour obama establish ties mullah...,0,0,0,1,0


In [ ]:
TrainData,TestData,TrainLabels,TestLabels = train_test_split(data_train,All_Data[1], test_size=0.2,random_state=1)